In [1]:
import pandas as pd
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Bidirectional, Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, SimpleRNN, Reshape
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import BinaryCrossentropy
import tensorflow as tf
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

2023-12-09 12:55:19.037771: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load training data from train.csv
train_data = pd.read_csv('train.csv')

# Load testing data from test.csv
test_data = pd.read_csv('test.csv')

# Assuming 'text' is the column name containing the text data in your CSV files
# Clean and preprocess the text column in the training data
nltk.download('punkt')
nltk.download('stopwords')

# Remove special characters, tokenize, and remove stopwords
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Remove special characters, tokenize, and remove stopwords
    text = text.lower()  # Convert text to lowercase
    tokens = word_tokenize(text)  # Tokenize the text
    clean_tokens = [token for token in tokens if token.isalpha() and token not in stop_words]  # Remove non-alphabetic tokens and stopwords
    return " ".join(clean_tokens)  # Join tokens back into a single string

# Apply the preprocessing function to the 'text' column in training and testing data
train_data['clean_text'] = train_data['text'].apply(preprocess_text)
test_data['clean_text'] = test_data['text'].apply(preprocess_text)

# Separate features and target for training and testing
X_train = train_data['clean_text']
y_train = train_data['target']

X_test = test_data['clean_text']

# Read the predicted file from our Bayes to get a y_test
df_test = pd.read_csv('test_predictions_cv.csv')
df_groundtruth = pd.read_csv('ground_truth.csv')

# Merge the DataFrames on 'id' column to have a single DataFrame for comparison
merged_df = pd.merge(df_test, df_groundtruth, on='id', suffixes=('_predicted', '_groundtruth'))
y_test = merged_df.iloc[:, 2]

[nltk_data] Downloading package punkt to
[nltk_data]     /home/cse478/mgriffin13/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cse478/mgriffin13/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Tokenize the text data
max_words = 20000  # Define the maximum number of words in your vocabulary
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

# Convert text to sequences and pad sequences for uniform length
maxlen = 100  # Define the maximum length of sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen)
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen)

# Create the RNN model
embedding_dim = 50  # Define the embedding dimension
num_classes = 2  # Assuming binary classification

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=maxlen))
model.add(SimpleRNN(64))  # Adjust the number of units as needed
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='sigmoid'))  # Use 'sigmoid' for binary classification otherwise 'softmax'

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 10  # Define the number of epochs
model.fit(X_train_pad, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test_pad, y_test))


2023-12-09 12:55:23.472257: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/10
238/238 [==============================] - 12s 44ms/step - loss: 0.5628 - accuracy: 0.7027 - val_loss: 0.4603 - val_accuracy: 0.7842
Epoch 2/10
238/238 [==============================] - 8s 34ms/step - loss: 0.2465 - accuracy: 0.9095 - val_loss: 0.5593 - val_accuracy: 0.7778
Epoch 3/10
238/238 [==============================] - 8s 32ms/step - loss: 0.1103 - accuracy: 0.9660 - val_loss: 0.5625 - val_accuracy: 0.7803
Epoch 4/10
238/238 [==============================] - 8s 32ms/step - loss: 0.0684 - accuracy: 0.9779 - val_loss: 0.6366 - val_accuracy: 0.7757
Epoch 5/10
238/238 [==============================] - 8s 34ms/step - loss: 0.0542 - accuracy: 0.9779 - val_loss: 0.6489 - val_accuracy: 0.7708
Epoch 6/10
238/238 [==============================] - 8s 34ms/step - loss: 0.0399 - accuracy: 0.9833 - val_loss: 0.7231 - val_accuracy: 0.7726
Epoch 7/10
238/238 [==============================] - 8s 33ms/step - loss: 0.0372 - accuracy: 0.9829 - val_loss: 1.0293 - val_accuracy: 0.750

In [4]:
model.evaluate(X_test_pad, y_test)

102/102 [==============================] - 1s 8ms/step - loss: 0.8448 - accuracy: 0.7668


[0.8447743654251099, 0.7667790651321411]

In [5]:
from sklearn.metrics import precision_score, recall_score, classification_report

# Evaluate the model on test data to get probabilities
y_pred_probs = model.predict(X_test_pad)

threshold = 0.5
y_pred_binary = (y_pred_probs[:, 1] > threshold).astype(int)

y_pred = np.array(y_pred_binary)

# Calculate precision
precision = precision_score(y_test, y_pred)

# Calculate recall
recall = recall_score(y_test, y_pred)

# Print precision and recall
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

# Classification report with precision, recall, F1-score for each class
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

102/102 [==============================] - 1s 8ms/step
Precision: 0.7767
Recall: 0.6526

Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.86      0.81      1861
           1       0.78      0.65      0.71      1402

    accuracy                           0.77      3263
   macro avg       0.77      0.76      0.76      3263
weighted avg       0.77      0.77      0.77      3263

